## Set Up

In [1]:
# Set Up 
import pandas as pd
import numpy as np
import dataframe_image as dfi
import joblib

from matplotlib import rcParams 
from matplotlib import pyplot as plt

rcParams['figure.figsize'] = [15, 5]
rcParams['font.size'] = 15
rcParams['axes.spines.top'] = False
rcParams['axes.spines.right'] = False
rcParams['figure.autolayout'] = True

In [2]:
import os, requests


fname = []
for j in range(3):
    fname.append('steinmetz_part%d.npz'%j)
url = ['https://osf.io/agvxh/download']
url.append('https://osf.io/uv3mw/download')
url.append('https://osf.io/ehmw2/download')

for i in range(len(url)):
    if not os.path.isfile(fname[i]):
        try:
            r = requests.get(url[i])
        except requests.ConnectionError:
            print("Data could not download!")
        else:
            if r.status_code != requests.codes.ok:
                print("Data could not download!")
            else:
                with open(fname[i], "wb") as fid:
                    fid.write(r.content)
                    

steinmetz_data = np.array([])
for i in range(len(fname)):
    steinmetz_data = np.hstack((steinmetz_data, np.load('/Users/sophiabatchelor/Code/SteinmetzAnalyses/steinmetz_part%d.npz'%i, allow_pickle=True)['dat']))

In [3]:
session_data = steinmetz_data[11]
num_timebins = session_data['spks'].shape[2]
labels = session_data['response'] # right - nogo - left (-1, 0, 1)
regions = np.unique(session_data['brain_area'])
spikes_in_a_region = {}
for region in regions:
    spikes_in_a_region[region] = session_data['spks'][np.where(session_data['brain_area']==region)]

In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

In [5]:
X = labels != 0 # boolean true 
target_regions = spikes_in_a_region.keys()
scores = np.zeros((len(target_regions),num_timebins))
for target_regions,(_, spikes) in enumerate(spikes_in_a_region.items()):
    for t in range(num_timebins):
        Y = spikes[:,:,t].T

In [9]:
X.shape

(340,)

In [8]:
Y.shape

(340, 100)

In [19]:
newX = X.reshape(-1, 1)

In [20]:
newX.shape

(340, 1)

In [11]:
# working on raising error messages 
if X.shape == Y.shape:
    try:
        X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.25,random_state=0)
        logistic_regression= LogisticRegression()
        logistic_regression.fit(X_train,y_train)
        y_pred=logistic_regression.predict(X_test)
    except ValueError:
        print("X and Y are not the same shape")

In [21]:
X_train,X_test,y_train,y_test = train_test_split(newX,Y,test_size=0.25,random_state=0)

In [22]:
logistic_regression= LogisticRegression()
logistic_regression.fit(X_train,y_train)
y_pred=logistic_regression.predict(X_test)

ValueError: Expected 2D array, got 1D array instead:
array=[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0.
 1. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 0.
 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1.
 1. 0. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1.
 0. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 1. 1. 0. 1. 1.].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [23]:
model = LogisticRegression(penalty='l2',multi_class='ovr',solver='liblinear')
spikes_in_a_region.keys, y, num_timebins = session_data
target_regions = spikes_in_a_region.keys()
scores = np.zeros((len(target_regions),num_timebins))
for target_regions,(_, spikes) in enumerate(spikes_in_a_region.items()):
    for dt in range(num_timebins):
        X = spikes[:,:,dt].T
        score = cross_val_score(model, X, y, cv=5)
        scores[region,dt] = np.mean(score)
    print(scores)

ValueError: too many values to unpack (expected 3)